### Loading necessary packages and preparing kernel.

In [1]:
from fastai.tabular.all import *
import pandas as pd
from itertools import chain

### Reading data

In [2]:
data = pd.read_excel('DeepTubeWell_Data.xlsx', sheet_name = 'Sheet1')
data.head()

,well_id,well,Y,X,Y_UTM,X_UTM,Material,Top Elevation,Thickness,Bottom Elevation,...,Well Size (mm),Q_lpm,permiability_cm/s,transmissivity_cm2/s,static water level_m,drawdown_m,screenlength_m,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,3,BalajuPark Well,27.73411,85.30196,3068903.385,332622.4245,Top Soil,1311.0,0.5,NaN,...,NaN,600.0,0.00263,NaN,83.9,4.05,NaN,NaN,Facies,Material
1,3,BalajuPark Well,27.73411,85.30196,3068903.385,332622.4245,Weathered Soil,NaN,4.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Clay
2,3,BalajuPark Well,27.73411,85.30196,3068903.385,332622.4245,Colluvium Deposite,NaN,60.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,silt
3,3,BalajuPark Well,27.73411,85.30196,3068903.385,332622.4245,Highly saturated fractured limestone,NaN,80.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,sand
4,3,BalajuPark Well,27.73411,85.30196,3068903.385,332622.4245,Partially Saturated Limestone,NaN,15.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,gravel


### Extracting first row of every well.

In [3]:
# data['well_id'].unique()

In [4]:
screened_data = pd.DataFrame()
for id in data['well_id'].unique():
    screened_data = pd.concat([screened_data, data[data['well_id'] == id].iloc[:1, :] ])

screened_data.head()

,well_id,well,Y,X,Y_UTM,X_UTM,Material,Top Elevation,Thickness,Bottom Elevation,...,Well Size (mm),Q_lpm,permiability_cm/s,transmissivity_cm2/s,static water level_m,drawdown_m,screenlength_m,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,3,BalajuPark Well,27.734110,85.301960,3068903.385,332622.4245,Top Soil,1311.0,0.5,NaN,...,NaN,600.0,0.002630,NaN,83.9,4.05,NaN,NaN,Facies,Material
6,7,Bansbari Bhaktapur Well,27.689220,85.457320,3063728.237,347876.3542,Top Soil,1370.0,1.0,NaN,...,NaN,400.0,0.000014,NaN,51.2,11.35,36.0,NaN,NaN,NaN
43,111,"Bansbari, Maharajgunj Tube Well",27.747077,85.345513,3070281.641,336935.4792,Top soil,1358.0,11.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,19,Bode solar tube well,27.694890,85.396140,3064433.453,341850.6854,Clay,1359.0,8.0,NaN,...,NaN,600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,20,Bode Treatment Plant,27.696740,85.393680,3064641.583,341610.7512,Black Clay,1324.0,4.9,NaN,...,NaN,600.0,NaN,NaN,NaN,NaN,60.0,NaN,NaN,NaN


### Filtering out rows containing no discharge values.

In [5]:
screened_data = screened_data[screened_data['Q_lpm'].notna()]
screened_data.head()

,well_id,well,Y,X,Y_UTM,X_UTM,Material,Top Elevation,Thickness,Bottom Elevation,...,Well Size (mm),Q_lpm,permiability_cm/s,transmissivity_cm2/s,static water level_m,drawdown_m,screenlength_m,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,3,BalajuPark Well,27.73411,85.30196,3068903.385,332622.4245,Top Soil,1311.0,0.5,NaN,...,NaN,600.0,0.002630,NaN,83.9,4.05,NaN,NaN,Facies,Material
6,7,Bansbari Bhaktapur Well,27.68922,85.45732,3063728.237,347876.3542,Top Soil,1370.0,1.0,NaN,...,NaN,400.0,0.000014,NaN,51.2,11.35,36.0,NaN,NaN,NaN
54,19,Bode solar tube well,27.69489,85.39614,3064433.453,341850.6854,Clay,1359.0,8.0,NaN,...,NaN,600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,20,Bode Treatment Plant,27.69674,85.39368,3064641.583,341610.7512,Black Clay,1324.0,4.9,NaN,...,NaN,600.0,NaN,NaN,NaN,NaN,60.0,NaN,NaN,NaN
117,21,Bode Tubewell Dugwell Compound,27.70280,85.39604,3065309.970,341852.2304,Top soil,1317.0,2.0,NaN,...,NaN,500.0,0.003900,NaN,41.6,14.20,66.0,NaN,NaN,NaN


In [32]:
qdata = screened_data.reset_index()[['well ','X_UTM','Y_UTM', 'Q_lpm']]
qdata

,well,X_UTM,Y_UTM,Q_lpm
0,BalajuPark Well,332622.4245,3068903.385,600.0
1,Bansbari Bhaktapur Well,347876.3542,3063728.237,400.0
2,Bode solar tube well,341850.6854,3064433.453,600.0
3,Bode Treatment Plant,341610.7512,3064641.583,600.0
4,Bode Tubewell Dugwell Compound,341852.2304,3065309.970,500.0
...,...,...,...,...
79,Sitapaila well,330476.6749,3066710.112,200.0
80,Suichataar well,330309.6899,3064993.617,350.0
81,Sukedhara well,337060.4114,3068666.772,400.0
82,Swayombhu Geeta Mandir,330923.4266,3066921.091,250.0


In [34]:
qdata.to_csv('cleaned-qdata.csv', index =False)

In [6]:
# pd.DataFrame([screened_data['well']])